# Biostat M280 Homework 4
Sarah Ji

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


## Q1 Solution: 
 
>First we should plug in the pi(p) formula in the f(x) formula and then combine like terms.

 $$
\int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p} 
= \int_{\Delta_d}\binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}d \mathbf{p}
$$
 
>Then we can use the kernel trick to integrate the first pdf to 1 and change the coefficients accordingly.
 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} d\mathbf{p}
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}\frac{\Gamma(|\alpha|+|\mathbf{x}|)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}\int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

> Note this is a density of a Dirchlet pdf and so the integral evalutes to 1, giving us: 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)} 1
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$

$$
= f(\mathbf{x} \mid \alpha)
$$

## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

## Q2 Solution:

The log likelihood is a non-concave function. 

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

## Q3 Solution:

In [14]:
using SpecialFunctions
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    term1 = 0.0
    Logpdf = 0.0
    sumalpha = sum(α)
    sumx = sum(x)
    for i in 1:length(x)
        if α[i] == 0 && x[i] == 0
            Logpdf += 0
        elseif α[i] == 0 && x[i] > 0
            Logpdf += - Inf
        else
        term1 += lgamma(α[i] + x[i]) - lgamma(α[i]) - lfact(x[i])
        Logpdf = term1 + lfact(sumx) + lgamma(sumalpha) - lgamma(sumx + sumalpha)
    end
    end
    return Logpdf
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

## Q4 Solution:

In [15]:
traindata = readcsv("optdigits.tra", Int)
digit = traindata[:, end]
X = copy(traindata[:, 1:64])
counts = traindata[:, 1:64]'

64×3823 Array{Int64,2}:
  0   0   0   0   0   0   0   0   0  …   0   0   0   0   0   0   0   0   0
  1   0   0   0   0   0   0   0   0      0   0   1   0   0   0   0   0   0
  6  10   8   0   5  11   1   8  15      9   9  10   6   5   0   3   6   2
 15  16  15   3  14  16  11  10   2     16  16  16  16  13   1  15  16  15
 12   6  16  11   4  10  13   8  14      6  12  16  11  11  12   0   2  16
  1   0  13  16   0   1  11   7  13  …   0   1   4   0   2   1   0   0  13
  0   0   0   0   0   0   7   2   2      0   0   0   0   0   0   0   0   1
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  7   7   1   0   0   4   0   1   0      2   3   8   1   2   0   0   0   0
 16  16  11   5  13  16   9  15  16  …  14  16  13  16  15   0  11  15   3
  6   8   9  16   8  10  14  14  15     16  10  12   2   6  14  14  10   7
  6  16  11  11   0  15   6  12  12     16  15  16  12   5  10   0   0  10
 

In [3]:
digit

3823-element Array{Int64,1}:
 0
 0
 7
 4
 6
 2
 5
 5
 0
 8
 7
 1
 9
 ⋮
 1
 4
 4
 8
 9
 3
 9
 9
 4
 6
 6
 7

In [4]:
eachlogpdf = dirmult_logpdf(counts, ones(64))

3823-element Array{Float64,1}:
 -165.188
 -176.23 
 -167.774
 -165.564
 -157.79 
 -176.071
 -158.423
 -159.258
 -174.302
 -178.407
 -171.294
 -169.383
 -175.753
    ⋮    
 -160.49 
 -158.633
 -156.935
 -171.975
 -177.638
 -169.735
 -158.423
 -159.465
 -159.877
 -169.735
 -173.149
 -155.411

Note that each observation is independent, so the overall log likelihood is the sum of each of the log pdf's in the `eachlogpdf` vector above.

In [5]:
Log_Likelihood = sum(eachlogpdf)


-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

## Q5 Solution:

>Let $\psi(x) = \frac{d}{dx}[ \ln \Gamma(x)]$ be the digamma function of x.
Let $\psi_1(x) = \frac{d}{dx}[\psi(x)]$ be the trigamma function of x.

>The Score function, $\nabla L(\alpha) = \frac{d}{d\alpha_j}L(\alpha) = 0 + \sum_{i=1}^n[\psi(\alpha_j + x_{ij}) - \psi(\alpha_j)] - \sum_{i=1}^n[\psi(|\alpha| + |x_i|) - \psi(|\alpha|)]$


>The observed information matrix $-d^2L(\alpha)$ has different forms on the diagonals and on the off diagonals.
<br>
>On the diagonals (where i = j):
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \sum_{i=1}^n[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})] - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$
<br>


>On the off diagonals (where i $\ne$ j)
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$

>Now we can see that the observed information matrix has a special structure of a diagonal plus an easy low-rank matrix which is rank 1.
$$
-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \mathbf{D} - c\mathbf{1}\mathbf{1}^T
$$

$$
d_j = \sum_{i=1}^n\Big[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})\Big]
$$

$$
c = \sum_{i=1}^n\Big[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|) \Big]
$$

<br>
>The Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ is difficult to compute, which highlights why Fisher scoring method is inefficient for computing MLE in this example. 

$$
-E\Big[\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) \Big] = E[\mathbf{D} - c\mathbf{1}\mathbf{1}^T]
$$

$$
E[d_j] = \sum_{i = 1}^n\sum_{k=0}^{|x_i|-1}\frac{P(X_{ij} > k)}{(\alpha_j + k)^2}
$$

Where $X_{ij}$ is the $j^{th}$ element of the $i^{th}$ observation of $X$, so it is a beta binomial distribution
and you have to evaluate the beta binomial distribution percentile function to get the tail probabilities for each observation.

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

## Q6 Solution:

No, the observed information matrix is not always positive definite. Some remedies we can make to it involve taking the expectation, resulting in swapping out the observed information matrix for the expected information matrix (Fisher Scoring). However, this remedy may not be the best case in this scenario, as taking expectation of the observed information matrix may be difficult. Thus, we use the Sherman Morrison formula from HW 1 to use the special structure of the observed information matrix, a diagonal plus a rank one update. 

>The observed information matrix has a special structure, a diagonal matrix plus a low rank matrix so we can use the Woodbury formula for the evaluation of the Newton direction. 

>In the case where the matrix fails to be diagonal we can add a positive matrix $\eta \mathbf{I}$, where $\eta$ > 0 to make it become positive definite.  

>From the Sherman-Morrison formula and the determinant formula from HW1:

$$
det(\mathbf{D} - c\mathbf{1}\mathbf{1}^T) = det(\mathbf{D})det(\mathbf{I} + c\mathbf{1}^T\mathbf{D}^{-1}\mathbf{1})
$$

$$
= (\prod_{i = 1}^d d_j)(1 - c \sum^d_{j=1}d_j^{-1})
$$
> In order to have a positive definite matrix, the diagonals and the determinant must be positive! So we check that:
$(1 - c \sum^d_{j=1}d_j^{-1}) > 0$


$$
1 > c\sum^d_{j=1}d_j^{-1} \implies c < \cfrac{1}{\sum^d_{j=1}d_j^{-1}}
$$


>We find that the observed information matrix is positive definite when $c < \cfrac{1}{\sum_{i = 1}^d d_j^{-1}}$.

## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)

### Q7 Solution:

Let $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$. Then, we can begin by noting the first and second moments of this distribution. 

>The first moment of the Dirichlet Distribution is:
$$E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d$$
<br>

>The second moment of the Dirichlet Distribution is:
$$E[P_j^2] = \frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha| + 1)}, 1 \le j \le d$$
<br>

Since we know $E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d,$ we know that $\alpha_j = E[P_j]*|\alpha|.$ Now we will use these two moments to first find the value of $\hat|\alpha|$ to find a good estimate for the starting value $\hat\alpha_j$
<br>

>Let $Z = \sum_{j=1}^d \frac{E[P_j^2]}{E[P_j]},$ then we have:

$$Z = \sum_{j = 1}^d \cfrac{E(p_j^2)}{E(p_j)} = \sum_{j = 1}^d \cfrac{\frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha|+ 1)}}{\frac{\alpha_j}{|\alpha|}} = \sum_{j=1}^d \frac{\alpha_j + 1}{|\alpha| + 1} = \frac{|\alpha| + \sum_{j=1}^d 1}{|\alpha| + 1} =  \cfrac{|\alpha| + d}{|\alpha| + 1}$$
<br>
>Now we want to solve for $|\alpha|$:
<br>
$$ Z(|\alpha| + 1) = |\alpha| + d$$
<br>
$$Z|\alpha| + Z = |\alpha| + d$$
<br>
$$Z|\alpha| - |\alpha| = d - Z$$
<br>
$$|\alpha|(Z - 1) = d - Z$$
<br>
$$|\alpha| = \frac{d - Z}{Z - 1}$$
<br>

> Recall that we are estimating Z by $\hat{Z}$:

$$\hat{Z} = \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}$$
<br>

>Now we can estimate $\hat{|\alpha|}$ using the estimates for $\hat{Z}$:
<br>

$$\hat{|\alpha|} = \frac{d - \hat{Z}}{\hat{Z} - 1}
$$
<br>

$$ = \frac{d - \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}}{\sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})} - 1}
$$
<br>

>Now we recall that $E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d,$ which means $\alpha_j = E[P_j]*|\alpha|.$ Using our method of moments estimate of $\hat{|\alpha|}$, we get:

$$\hat{\alpha_j} = E[P_j]*\hat{|\alpha|}
$$
<br>
$$ = \frac{(\sum_i^n\frac{x_{ij}}{|x_i|})}{n}*\hat{|\alpha|}
$$
<br>
$$ = \frac{(\sum_i^n\frac{x_{ij}}{|x_i|})}{n}*\frac{d - \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}}{\sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})} - 1}
$$

In [16]:
## code for finding initial alpha
function initial_alpha(X::Matrix{Int})
    n, d = size(X)    #@show n #64    #@show d #3823
    alpha0 = zeros(d) #creates 64x1 matrix
    tolerance = 0.0001
    
    khat = 0
    knumerator = 0
       
    sumcol = sum(X,1)
    sumrow = sum(X,2)
    
    for j in 1:d #each row   
    knumerator = 0 #reset each value
        
        if sumcol[j] < tolerance #check to see if there is data in the column
            continue
        end
        
        for i in 1:n #for each column
            knumerator += (X[i,j] / sumrow[i])^2 #numerator in the k equation
            alpha0[j] += (X[i,j] / sumrow[i]) #denominator in the k equation, and second part of the equation for aj0
        end
        
        khat += (knumerator / alpha0[j]) #calculating khat
        alpha0[j] /= n #divide by n in front of the sigma

    end
    alpha0 *= ((d - khat) / (khat - 1))
    
    return alpha0
    
end

alpha0 = initial_alpha(X)

64-element Array{Float64,1}:
 0.0        
 0.0714198  
 1.30543    
 2.83281    
 2.75176    
 1.32356    
 0.336655   
 0.0343316  
 0.000461199
 0.463798   
 2.52787    
 2.8072     
 2.53299    
 ⋮          
 2.33097    
 2.20214    
 0.894695   
 0.0353529  
 5.71236e-5 
 0.066766   
 1.39754    
 2.8661     
 2.744      
 1.59808    
 0.506003   
 0.0475158  

In [68]:
function dirmult_newton(X) 
    maxiters = 100
    tolfun = 1e-6
    orign, origd = size(X)
    #first reduce to where the obs are 0
    nonzeroobs = find(sum(X, 1) .> 0.01) 
    #remove those rows in X
    X = copy(X[:, nonzeroobs])
    Xt = copy(X.')
    # set default starting point from Q7
    α0 = initial_alpha(X)
    rowsumX = sum(X, 2)
    n, d = size(X)
    dga = zeros(d)
    score = zeros(d)
    αhat = copy(α0)
    ndir = zeros(d)
    triarow = 0.0
    loglold = sum(dirmult_logpdf(Xt, αhat))
    # Newton loop
    for iter in 1:maxiters
        rowsumα = sum(αhat)
        score = zeros(d)       
        # evaluate gradient (score)
        dga = digamma.(αhat)
        dgarow = digamma.(rowsumα)
        for j in 1:d
            for i in 1:n
                score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
            end
        end
        # approximated observed information matrix

        D = zeros(d)
        c = 0.0
        triga = trigamma.(αhat)
        for j = 1:d
            for i = 1:n
                D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
                c += -trigamma.(rowsumα + rowsumX[i]) + triarow
            end
        end

        #need to make sure that the condition for c holds for it to be p.d.
        invD = (1 ./ D)
        if !(c < (1 / (sum(invD))))
            c = (1 / sum(invD)) * 0.98
        end

        # compute Newton's direction
        #use Sherman-Morrison inverse formula
        #first part is invD * score
        #second is 1 / (1 - c*sum(Dinv)) * Dinv' * Dinv * c

        for j in 1:d
            ndir[j] = invD[j] * score[j]
        end
        ndir += (((1/c - sum(invD)) * (invD * invD')) * score)

        # line search loop
        step = 1
        αupdate = 0.0
        logl = 0.0
        loglupdate = 0.0
        for lsiter in 1:10
            αupdate = αhat + step * ndir
            loglupdate = sum(dirmult_logpdf(Xt, αupdate))
            # step halving
            if (loglupdate < loglold)
                step = step / 2
            end
        end
        αhat = copy(αupdate)
        logold = copy(logl)
        logl = copy(loglupdate)

        # check convergence criterion
        if abs(logl - loglold) < (tolfun * (abs(loglold) + 1))
            break;
        end
    end

    # compute logl, gradient, Hessian from final iterate 

    #logl already computed. 
    rowsumα = sum(αhat)
    score = zeros(d)       
    # evaluate gradient (score)
    dga = digamma.(αhat)
    dgarow = digamma.(rowsumα)
    for j in 1:d
        for i in 1:n
            score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
        end
    end

    #evaluating the observed info matrix D is the diagonal and c is the off constant. 
    D = zeros(d)
    c = 0.0
    triga = trigamma.(αhat)
    triarow = trigamma.(rowsumα)
    for j = 1:d
        for i = 1:n
            D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
            c += -trigamma.(rowsumα + rowsumX[i]) + triarow
        end
    end


    hessian = -1 * (Diagonal(D) + (c * ones(d, d)))
    
    αestimate = zeros(origd)
    αestimate[nonzeroobs] = αhat[1:d]
    return αestimate, hessian
end

dirmult_newton (generic function with 1 method)

In [17]:
function dirmult_newton_MLE(X) 
    maxiters = 100
    tolfun = 1e-6
    orign, origd = size(X)
    #first reduce to where the obs are 0
    nonzeroobs = find(sum(X, 1) .> 0.01) 
    #remove those rows in X
    X = copy(X[:, nonzeroobs])
    Xt = copy(X.')
    # set default starting point from Q7
    α0 = initial_alpha(X)
    rowsumX = sum(X, 2)
    n, d = size(X)
    dga = zeros(d)
    score = zeros(d)
    αhat = copy(α0)
    ndir = zeros(d)
    triarow = 0.0
    loglold = sum(dirmult_logpdf(Xt, αhat))
    # Newton loop
    for iter in 1:maxiters
        rowsumα = sum(αhat)
        score = zeros(d)       
        # evaluate gradient (score)
        dga = digamma.(αhat)
        dgarow = digamma.(rowsumα)
        for j in 1:d
            for i in 1:n
                score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
            end
        end
        # approximated observed information matrix

        D = zeros(d)
        c = 0.0
        triga = trigamma.(αhat)
        for j = 1:d
            for i = 1:n
                D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
                c += -trigamma.(rowsumα + rowsumX[i]) + triarow
            end
        end

        #need to make sure that the condition for c holds for it to be p.d.
        invD = (1 ./ D)
        if !(c < (1 / (sum(invD))))
            c = (1 / sum(invD)) * 0.95
        end

        # compute Newton's direction
        #use Sherman-Morrison inverse formula
        #first part is invD * score
        #second is 1 / (1 - c*sum(Dinv)) * Dinv' * Dinv * c

        for j in 1:d
            ndir[j] = invD[j] * score[j]
        end
        ndir += (((1/c - sum(invD)) * (invD * invD')) * score)

        # line search loop
        step = 1
        for i in 1:d
            if ndir[i] < 0
                step = min(- αhat[i] / ndir[i] * 0.95, step)
            end
        end
        αupdate = 0.0
        logl = 0.0
        loglupdate = 0.0
        for lsiter in 1:10
            αupdate = αhat + step * ndir
            loglupdate = sum(dirmult_logpdf(Xt, αupdate))
            # step halving
            if (loglupdate < loglold)
                step = step / 2
            end
        end
        αhat = copy(αupdate)
        logold = copy(logl)
        logl = copy(loglupdate)

        # check convergence criterion
        if abs(logl - loglold) < (tolfun * (abs(loglold) + 1))
            break;
        end
    end

    # compute logl, gradient, Hessian from final iterate 

    #logl already computed. 
    rowsumα = sum(αhat)
    score = zeros(d)       
    # evaluate gradient (score)
    dga = digamma.(αhat)
    dgarow = digamma.(rowsumα)
    for j in 1:d
        for i in 1:n
            score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
        end
    end

    #evaluating the observed info matrix D is the diagonal and c is the off constant. 
    D = zeros(d)
    c = 0.0
    triga = trigamma.(αhat)
    triarow = trigamma.(rowsumα)
    for j = 1:d
        for i = 1:n
            D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
            c += -trigamma.(rowsumα + rowsumX[i]) + triarow
        end
    end


    hessian = -1 * (Diagonal(D) + (c * ones(d, d)))
    
    αestimate = zeros(origd)
    αestimate[nonzeroobs] = αhat[1:d]
    return αestimate
end

dirmult_newton_MLE (generic function with 1 method)

In [8]:
dirmult_newton_MLE(X)

64-element Array{Float64,1}:
 0.0        
 0.0805771  
 0.888962   
 2.08628    
 2.00262    
 0.759014   
 0.145656   
 0.0150775  
 0.000387974
 0.299796   
 1.7334     
 2.12607    
 1.85579    
 ⋮          
 1.64635    
 1.39108    
 0.456775   
 0.0271294  
 0.000129273
 0.0651677  
 0.913519   
 2.09549    
 1.92582    
 0.938188   
 0.230337   
 0.0276723  

In [12]:
αestimate_t, hessian_t = dirmult_newton(X)

([0.0, 0.0805771, 0.888962, 2.08628, 2.00262, 0.759014, 0.145656, 0.0150775, 0.000387974, 0.299796  …  0.456775, 0.0271294, 0.000129273, 0.0651677, 0.913519, 2.09549, 1.92582, 0.938188, 0.230337, 0.0276723], [-95935.9 -4305.26 … -4305.26 -4305.26; -4305.26 -9533.56 … -4305.26 -4305.26; … ; -4305.26 -4305.26 … -29853.3 -4305.26; -4305.26 -4305.26 … -4305.26 -2.748e5])

In [13]:
hessian_t

62×62 Array{Float64,2}:
 -95935.9   -4305.26  -4305.26  -4305.26  …  -4305.26   -4305.26  -4305.26   
  -4305.26  -9533.56  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -6257.36  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -6343.95     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26  …  -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26  …  -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26     -4305.26   -4305.26  -4305.26   
  -4305.26  -4305.26  -4305.26  -4305.26

In [14]:
αestimate_t

64-element Array{Float64,1}:
 0.0        
 0.0805771  
 0.888962   
 2.08628    
 2.00262    
 0.759014   
 0.145656   
 0.0150775  
 0.000387974
 0.299796   
 1.7334     
 2.12607    
 1.85579    
 ⋮          
 1.64635    
 1.39108    
 0.456775   
 0.0271294  
 0.000129273
 0.0651677  
 0.913519   
 2.09549    
 1.92582    
 0.938188   
 0.230337   
 0.0276723  

## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

In [18]:
traindata = readcsv("optdigits.tra", Int)

3823×65 Array{Int64,2}:
 0  1   6  15  12   1   0  0  0  7  16  …  0  0  0   6  14   7   1   0  0  0
 0  0  10  16   6   0   0  0  0  7  16     0  0  0  10  16  15   3   0  0  0
 0  0   8  15  16  13   0  0  0  1  11     0  0  0   9  14   0   0   0  0  7
 0  0   0   3  11  16   0  0  0  0   5     0  0  0   0   1  15   2   0  0  4
 0  0   5  14   4   0   0  0  0  0  13     0  0  0   4  12  14   7   0  0  6
 0  0  11  16  10   1   0  0  0  4  16  …  3  0  0  10  16  16  16  16  6  2
 0  0   1  11  13  11   7  0  0  0   9     0  0  0   1  13   5   0   0  0  5
 0  0   8  10   8   7   2  0  0  1  15     0  0  0   4  13   8   0   0  0  5
 0  0  15   2  14  13   2  0  0  0  16     0  0  0  10  12   5   0   0  0  0
 0  0   3  13  13   2   0  0  0  6  16     0  0  0   3  15  11   6   0  0  8
 0  0   6  14  14  16  16  8  0  0   7  …  0  0  0  10  12   0   0   0  0  7
 0  0   0   3  16  11   1  0  0  0   0     0  0  0   0   2  14  14   1  0  1
 0  0   0   4  13  16  16  3  0  0   8     0  0  0  

In [19]:
α_MLE = zeros(64, 10)
for digit in 0:9
data = X[find(traindata[:, end] .== digit), :]
α_MLE[:, digit + 1] = dirmult_newton_MLE(data)
end
α_MLE

64×10 Array{Float64,2}:
 0.0        0.0         0.0         …  0.0         0.0         0.0       
 0.027153   0.00801854  0.389006       0.139634    0.0879102   0.0764628 
 3.24571    0.513341    3.81971        2.49591     2.49451     1.53094   
 9.58928    2.0947      5.24924        5.05699     5.82549     3.70657   
 7.80525    3.05133     2.51317        5.44953     5.57461     3.73923   
 1.58007    1.31897     0.31109     …  4.46116     2.41974     1.51427   
 0.0458468  0.137636    0.0157885      1.71884     0.199532    0.298692  
 0.0        0.0         0.0            0.178885    0.0         0.0159489 
 0.0        0.0         0.0            0.0         0.00219596  0.0       
 0.676543   0.0572244   1.73504        0.254802    1.02163     0.667242  
 9.38799    1.03895     5.50464     …  3.17471     6.09556     4.02596   
 9.49482    3.23395     4.75728        3.98969     4.55733     3.35353   
 8.83415    4.09706     4.44696        4.16922     4.0813      3.29381   
 ⋮            

In [13]:
# read in the training and test data
train = readcsv("optdigits.tra", Int)
test = readcsv("optdigits.tes", Int)

p_hat = zeros(10)
n1 = size(train, 1)
n2 = size(test, 1)
probs = zeros( 10, size(test, 1))

# find prior probabilities for digits
for digit in 0:9
    p_hat[digit + 1] += sum(train[:, 65] .== digit) / n1
end

for digit in 0:9
    probs[digit + 1, :] = dirmult_logpdf(test[:, 1:64]', α_MLE[:, digit + 1])
end

# get posterior distribution
for j in 1:10
    probs[j,:] *= p_hat[j]
end

# get predicted digits based on largest log likelihood values
predicted = zeros(n2)
for i in 1:n2
    predicted[i] = (findmax(probs[:,i],1)[2][1] - 1)
end

# get the % of error
sum(test[:, 65] .!= predicted) / length(predicted)

0.1213132999443517

In [16]:
p_hat

10-element Array{Float64,1}:
 0.0983521
 0.101753 
 0.0993984
 0.101753 
 0.101229 
 0.0983521
 0.0986137
 0.101229 
 0.0993984
 0.0999215

In [17]:
probs

10×1797 Array{Float64,2}:
   -9.18952  -19.4321    -18.9039  …   -16.7766   -13.7951   -17.1748
  -17.2719    -9.16423   -12.2474      -11.1933   -13.8898   -14.4283
  -18.3713   -13.2906    -13.7059      -13.6755   -15.9799   -13.8357
  -17.3074   -14.2394    -16.8422      -14.5668   -13.3025   -14.041 
  -14.2874   -11.6727    -14.3059      -12.5735   -14.2097   -15.7324
  -14.034    -12.2124    -15.5329  …   -13.2366   -12.7971   -14.6144
 -Inf        -15.1889    -15.524      -Inf       -Inf        -14.2976
  -17.079    -14.3907   -Inf           -14.8126   -16.6913  -Inf     
  -14.9875   -11.6959    -12.7772      -10.5587   -12.9738   -12.365 
  -13.4609   -13.0306    -15.6665      -13.3276   -10.8463   -14.7031

In [10]:
traindata

3823×65 Array{Int64,2}:
 0  1   6  15  12   1   0  0  0  7  16  …  0  0  0   6  14   7   1   0  0  0
 0  0  10  16   6   0   0  0  0  7  16     0  0  0  10  16  15   3   0  0  0
 0  0   8  15  16  13   0  0  0  1  11     0  0  0   9  14   0   0   0  0  7
 0  0   0   3  11  16   0  0  0  0   5     0  0  0   0   1  15   2   0  0  4
 0  0   5  14   4   0   0  0  0  0  13     0  0  0   4  12  14   7   0  0  6
 0  0  11  16  10   1   0  0  0  4  16  …  3  0  0  10  16  16  16  16  6  2
 0  0   1  11  13  11   7  0  0  0   9     0  0  0   1  13   5   0   0  0  5
 0  0   8  10   8   7   2  0  0  1  15     0  0  0   4  13   8   0   0  0  5
 0  0  15   2  14  13   2  0  0  0  16     0  0  0  10  12   5   0   0  0  0
 0  0   3  13  13   2   0  0  0  6  16     0  0  0   3  15  11   6   0  0  8
 0  0   6  14  14  16  16  8  0  0   7  …  0  0  0  10  12   0   0   0  0  7
 0  0   0   3  16  11   1  0  0  0   0     0  0  0   0   2  14  14   1  0  1
 0  0   0   4  13  16  16  3  0  0   8     0  0  0  

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q10 Solution:

Dirichlet Multinomial is better for all digits.

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.

test set to get the prior probability distribution for p and then multiply each DMloglikelihood to get posterior distribution of each digit, plug in new image to this posterior to do classification